## Organización de las imágenes Inpaintadas (sin procesar) en el árbol de subdirectorios

Voy a aprovechar la estructura de directorios que ya tengo creada para el TFG y simplemente mover los ficheros para replicar esa misma estructura.

In [1]:
import os
import shutil

Primero vamos a obtener una lista con el nombre de las imágenes de cada subdirectorio, para después poder organizar las que tenemos disponibles en nuestro repositorio (de OCT no hace falta ya que simplemente copiaremos y pegaremos las imágenes del directorio del OCT de TFG a este). 

In [10]:
samsung_G1 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/Samsung/G1')
samsung_G2 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/Samsung/G2')
samsung_G3 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/Samsung/G3')
samsung_G4 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/Samsung/G4')
samsung_G5 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/Samsung/G5')

In [13]:
iphone_G1 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/iPhone/G1')
iphone_G2 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/iPhone/G2')
iphone_G3 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/iPhone/G3')
iphone_G4 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/iPhone/G4')
iphone_G5 = os.listdir('../TFG/CNN_and_Diabetic_Retinopathy/Datos/Processed Data/iPhone/G5')

Posteriormente debemos obtener un listado con todas las imágenes _inpaintadas_ de Samsung y de iPhone, para poder moverlas a la carpeta correspondiente.

In [22]:
samsung = os.listdir('Datos_INP/FOTOS Samsung')
iphone = os.listdir('Datos_INP/FOTOS iPhone')

Por último, recorremos todos los nombres de los archivos de Samsung y de iPhone _inpaintados_, y según a qué subdirectorio pertenezcan los movemos.

In [24]:
for i in samsung:
    if i in samsung_G1:
        shutil.move('Datos_INP/FOTOS Samsung/' + i, 'Images/Samsung/G1/' + i)
    elif i in samsung_G2:
        shutil.move('Datos_INP/FOTOS Samsung/' + i, 'Images/Samsung/G2/' + i)
    elif i in samsung_G3:
        shutil.move('Datos_INP/FOTOS Samsung/' + i, 'Images/Samsung/G3/' + i)
    elif i in samsung_G4:
        shutil.move('Datos_INP/FOTOS Samsung/' + i, 'Images/Samsung/G4/' + i)
    elif i in samsung_G5:
        shutil.move('Datos_INP/FOTOS Samsung/' + i, 'Images/Samsung/G5/' + i)

In [25]:
for i in iphone:
    if i in iphone_G1:
        shutil.move('Datos_INP/FOTOS iPhone/' + i, 'Images/iPhone/G1/' + i)
    elif i in iphone_G2:
        shutil.move('Datos_INP/FOTOS iPhone/' + i, 'Images/iPhone/G2/' + i)
    elif i in iphone_G3:
        shutil.move('Datos_INP/FOTOS iPhone/' + i, 'Images/iPhone/G3/' + i)
    elif i in iphone_G4:
        shutil.move('Datos_INP/FOTOS iPhone/' + i, 'Images/iPhone/G4/' + i)
    elif i in iphone_G5:
        shutil.move('Datos_INP/FOTOS iPhone/' + i, 'Images/iPhone/G5/' + i)

## Procesamiento Gaussiano de las imágenes

Una vez hemos dividido las imágenes inpaintadas en la estructura de directorios necesaria, vamos a aplicar a todas ellas el preprocesamiento gaussiano que resalta las características de la imagen.

In [26]:
import cv2
import PIL
import numpy as np

from PIL import Image

Primero definimos las dos funciones que van a aplicar el filtro de desenfoque Gaussiano y luego el 'troquelado' circular.

In [27]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

In [28]:
def circle_crop(img, sigmaX):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

A continuación aplicamos este procesado a todas las imágenes inpaintadas (tanto de Samsung, como de iPhone como de OCT).

In [33]:
for i in os.listdir('Images/Samsung/G1'):
    img = cv2.imread('Images/Samsung/G1/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/Samsung/G1/'+ i)

In [34]:
for i in os.listdir('Images/Samsung/G2'):
    img = cv2.imread('Images/Samsung/G2/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/Samsung/G2/'+ i)

In [35]:
for i in os.listdir('Images/Samsung/G3'):
    img = cv2.imread('Images/Samsung/G3/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/Samsung/G3/'+ i)

In [36]:
for i in os.listdir('Images/Samsung/G4'):
    img = cv2.imread('Images/Samsung/G4/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/Samsung/G4/'+ i)

In [37]:
for i in os.listdir('Images/Samsung/G5'):
    img = cv2.imread('Images/Samsung/G5/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/Samsung/G5/'+ i)

---------------------

In [38]:
for i in os.listdir('Images/iPhone/G1'):
    img = cv2.imread('Images/iPhone/G1/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/iPhone/G1/'+ i)

In [39]:
for i in os.listdir('Images/iPhone/G2'):
    img = cv2.imread('Images/iPhone/G2/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/iPhone/G2/'+ i)

In [40]:
for i in os.listdir('Images/iPhone/G3'):
    img = cv2.imread('Images/iPhone/G3/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/iPhone/G3/'+ i)

In [41]:
for i in os.listdir('Images/iPhone/G4'):
    img = cv2.imread('Images/iPhone/G4/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/iPhone/G4/'+ i)

In [42]:
for i in os.listdir('Images/iPhone/G5'):
    img = cv2.imread('Images/iPhone/G5/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/iPhone/G5/'+ i)

--------------

In [43]:
for i in os.listdir('Images/OCT/G1'):
    img = cv2.imread('Images/OCT/G1/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/OCT/G1/'+ i)

In [44]:
for i in os.listdir('Images/OCT/G2'):
    img = cv2.imread('Images/OCT/G2/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/OCT/G2/'+ i)

In [45]:
for i in os.listdir('Images/OCT/G3'):
    img = cv2.imread('Images/OCT/G3/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/OCT/G3/'+ i)

In [46]:
for i in os.listdir('Images/OCT/G4'):
    img = cv2.imread('Images/OCT/G4/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/OCT/G4/'+ i)

In [47]:
for i in os.listdir('Images/OCT/G5'):
    img = cv2.imread('Images/OCT/G5/'+ i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_t = circle_crop(img, 30)
    image = Image.fromarray(img_t, 'RGB')
    image.save('Images_Proc/OCT/G5/'+ i)